In [1]:
import json
from supernova import Supernova

### First, let's load in some data

In [2]:
# Let's load in some data
with open("/Users/QCaudron/Downloads/property/fake_addresses.json", "r") as f:
    x1, x2, y = json.load(f)

### Let's explore the data a little

In [3]:
# Explore the data a little
x1[:30]

['9020 Rancho Park Ct, Rancho Cucamonga, CA, 91729',
 '9020 Rancho Park Ct, Rancho Cucamonga, CA, 91729',
 '9020 Rancho Park Ct, Rancho Cucamonga, CA, 91729',
 '9020 Rancho Park Ct, Rancho Cucamonga, CA, 91729',
 '9020 Rancho Park Ct, Rancho Cucamonga, CA, 91729',
 '9020 Rancho Park Ct, Rancho Cucamonga, CA, 91729',
 '9020 Rancho Park Ct, Rancho Cucamonga, CA, 91729',
 '9020 Rancho Park Ct, Rancho Cucamonga, CA, 91729',
 '9020 Rancho Park Ct, Rancho Cucamonga, CA, 91729',
 '9020 Rancho Park Ct, Rancho Cucamonga, CA, 91729',
 '9020 Rancho Park Ct, Rancho Cucamonga, CA, 91729',
 '9020 Rancho Park Ct, Rancho Cucamonga, CA, 91729',
 '9020 Rancho Park Ct, Rancho Cucamonga, CA, 91729',
 '9020 Rancho Park Ct, Rancho Cucamonga, CA, 91729',
 '9020 Rancho Park Ct, Rancho Cucamonga, CA, 91729',
 '9020 Rancho Park Ct, Rancho Cucamonga, CA, 91729',
 '9020 Rancho Park Ct, Rancho Cucamonga, CA, 91729',
 '9020 Rancho Park Ct, Rancho Cucamonga, CA, 91729',
 '9020 Rancho Park Ct, Rancho Cucamonga, CA, 9

In [4]:
x2[:30]

['9020 Rancho Park Ct E, Rancho Cucamonga, CA, 91729',
 '9020 Rancho Park Ct, Rancho Cucamonga, CA, 91729',
 '9020 Rancho Park Ct S, Rancho Cucamonga, CA, 91729',
 '9020 Rancho Park Court, Rancho Cucamonga, 91729',
 '9020 Rancho Park Ct, Rancho Cucamonga, 91729',
 '501 Atlantic Freeway, Los Angeles, CA, 90022',
 '420 Garden Ave, Clearwater, FL, 33756',
 '12635 Saticoy St, North Hollywood, CA, 91605',
 '3000 Arden Way, Sacramento, CA, 95825',
 '1150 Foothill Blvd, La Canada Flintridge, 91011',
 '345 Chestnut Ave, Monrovia, CA, 91016',
 '3104 1030 , West Valley City, 84119',
 '10904 7th Ave, Miami, FL, 33168',
 '16209 25th Ave, Lynnwood, WA, 98037',
 '1801 Dixiana Rd, West Columbia, SC, 29172',
 '21421 68th Pkwy, Kent, WA, 98032',
 '4867 Harold Gatty Dr, Salt Lake City, UT, 84116',
 '8309 Interstate 35 Cir, AustinCir, 78753',
 '5532 Washington Blvd, Los Angeles, CA, 90016',
 '502 Shoreview Park Rd, Shoreview, MN, 55126',
 '1517 130th St, Gardena, CA, 90249',
 '2050 Duane Ave, Santa Clara

In [5]:
y[:30]

[True,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True]

The data in comes in blocks of 25. Each block contains the same address at each element of `x1`. In `x2`, the first five are variations of that address, and the last twenty are random other addresses. As such, the entries of `y` are also blocks of 25, with the first five being `True` and the last twenty being `False`.

This is fake(ish) data. The original addresses are from EDP, from Dave's dump. The variants are very simple fake mutations we put together. The result ( `x1`, `x2`, and `y` ) is a labelled dataset.

### Let's spin up a deduper

In [6]:
# For Levenshtein distances
from fuzzywuzzy import fuzz


# Our features
def street_number_same(x1, x2):
    return x1.split(" ")[0] == x2.split(" ")[0]
        
def same_city(x1, x2):
    return x1.split(", ")[1] == x2.split(", ")[1]

def fuzzy(x1, x2):
    return fuzz.ratio(x1, x2) / 100


# Our deduper object
class AddressDeduplicator(Supernova):

    # Set the filename and set the model
    filename = "address_deduper"

    # Define the set of features
    featureset = [
        street_number_same,
        same_city,
        fuzzy
    ]

In [13]:
# Instantiate and train on the first thousand
deduper = AddressDeduplicator()
deduper.fit(x1[:1000], x2[:1000], y[:1000])

Fitting 5 folds for each of 1 candidates, totalling 5 fits


/Users/qcaudron/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/qcaudron/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/qcaudron/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/qcaudron/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empt

Model score : 0.9970000000000001 +/- 0.008000000000000007


/Users/qcaudron/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/qcaudron/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/qcaudron/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/qcaudron/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empt

Looks good, we have a model score of 99.7%. Ignore all the warnings in pink, we'll be upgrading some packages so that those DeprecationWarnings go away.

### Testing the deduper

In [14]:
# Our known address "database"
boop = [
    "123 Main Street, Seattle, WA, 98101",
    "420 5th Ave, Seattle, WA, 98101",
    "123, Main Lane, Spokane, WA, 99999"
]

# Someone enters in "123 Main St, Seattle". Do we have this address in our DB ?

deduper.predict(boop, "123 Main St, Seattle")

array([[0.6960388 ],
       [0.00085097],
       [0.00093192]], dtype=float32)

The probability that "123 Main St, Seattle" is a dupe of "123 Main Street, Seattle, WA, 98101" comes back at 69.6%. The other scores are very small, under 1%. We've found a duplicate.

### Loading the deduper from scratch

We're trying to run this on another box. At this point, Supernova cannot save the featureset, so we have to rebuild it.

In [15]:
# Our deduper object
class AddressDeduplicator(Supernova):

    # Set the filename and set the model
    filename = "address_deduper"

    # Define the set of features
    featureset = [
        street_number_same,
        same_city,
        fuzzy
    ]

Great, this has the same features as before ( that's essential ), and the same filename ( that's essential if you don't want to have to retrain the model ).

In [18]:
deduper2 = AddressDeduplicator()

In [19]:
deduper2.predict(boop, boop)

array([[9.9651611e-01, 6.9603878e-01, 1.5433135e-03],
       [6.9603878e-01, 9.9651611e-01, 8.2797377e-04],
       [1.5433135e-03, 8.2797377e-04, 9.9651611e-01]], dtype=float32)

We didn't have to retrain this one, because Supernova found a file on disk that matched the given filename. Instead of training our model, we just loaded the one we previously trained from disk. As such, we were able to to straight to `.predict`. Here, we passed in `boop` twice, and we therefore got back a `(len(boop), len(boop))`-shaped array. We see that the diagonals are very high ( 0.9965 ), whereas most of the rest is very small ( < 0.01 ).

However, note that entries at (0, 1) and (1, 0) are 0.696. This is high, and we wouldn't expect `boop[0]` and `boop[1]` to be duplicates. We probably need more ( better ) features. Comparing the cities is a pretty terrible idea, really. However, this was a quick example.